In [1]:
import pandas as pd
import re, os, string
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, average_precision_score,precision_score,f1_score,recall_score
import spacy

In [2]:
nlp_nl = spacy.load('nl_core_news_sm')

In [3]:
def clean_text(text):
    """Doc cleaning"""
    
    # Lowering text
    text = text.lower()
    
    # Removing punctuation
    text = "".join([c for c in text if c not in PUNCTUATION])
    
    # Removing whitespace and newlines
    text = re.sub('\s+',' ',text)
    
    # extract Noun
    noun_words = []
    for token in nlp_nl(text):
        taged_w = token.pos_
        if taged_w == 'NOUN':
            noun_words.append(token.lemma_)
            text = ' '.join(noun_words)
    
    return text

In [4]:
def get_stopwords_list(stop_file_path):
    """load stop words """
    
    with open(stop_file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return list(frozenset(stop_set))

In [5]:
def sort_coo(coo_matrix):
    """Sort a dict with highest score"""
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature, score
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [6]:
def get_keywords(vectorizer, feature_names, doc):
    """Return top k keywords from a doc using TF-IDF method"""

    #generate tf-idf for the given document
    tf_idf_vector = vectorizer.transform([doc])
    
    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(tf_idf_vector.tocoo())

    #extract only TOP_K_KEYWORDS
    keywords=extract_topn_from_vector(feature_names,sorted_items,TOP_K_KEYWORDS)
    
    return list(keywords.keys())

In [7]:
# Constants
PUNCTUATION = """!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~""" 
TOP_K_KEYWORDS = 10 # top k number of keywords to retrieve in a ranked document
STOPWORD_PATH = "../data/stopwords/nl_stopwords_list.txt"
PAPERS_PATH = "../data/test_data.csv"

In [8]:
data = pd.read_csv(PAPERS_PATH,encoding="unicode_escape")
data.head()

,title,chapter,content,gold_label,bl_label,w2v_label
0,Library Guide Google scholar,Google scholar,Wat is Google Scholar?\nGoogle Scholar biedt e...,Selecteren informatiebronnen en zoeksystemen,Selecteren informatiebronnen en zoeksystemen,Aard en verschijningsvormen van informatie/data
1,Library Guide Healthy Ageing,Healthy Ageing,Systematisch zoeken\nIn deze library guide kri...,Plannen en zoeken,Informatievaardigheden,Manage proces en zoekresultaten
2,Library Guide Refworks,Reforks,Wat is RefWorks?\nBeheer en verwerk je bronnen...,Managen proces en zoekresultaten,Manage proces en zoekresultaten,Manage proces en zoekresultaten
3,Outreach de focus,Hoe wat waarom,"Na het voltooien van je scriptie, hoeft het we...",Valoriseren van bevindingen (outreach),Valoriseren van bevindingen (outreach),Waarde van informatie/data
4,Hoe herken je fake news,NaN,en heel erg om de nieuws uit met name rusland ...,Beoordelingscriteria relevantie/ betrouwbaarheid,Beoordelen van resultaten (kritisch),Beoordelen van resultaten (kritisch)


In [9]:
data.dropna(subset=['content'], inplace=True)

In [10]:
data['content'] = data['content'].apply(clean_text)
data.head()

,title,chapter,content,gold_label,bl_label,w2v_label
0,Library Guide Google scholar,Google scholar,manier lijn literatuur plek discipline bron ar...,Selecteren informatiebronnen en zoeksystemen,Selecteren informatiebronnen en zoeksystemen,Aard en verschijningsvormen van informatie/data
1,Library Guide Healthy Ageing,Healthy Ageing,uitleg aanpak gebied ageing tabblad zoekstrate...,Plannen en zoeken,Informatievaardigheden,Manage proces en zoekresultaten
2,Library Guide Refworks,Reforks,beheer verwerken refwork refwork citatie logbo...,Managen proces en zoekresultaten,Manage proces en zoekresultaten,Manage proces en zoekresultaten
3,Outreach de focus,Hoe wat waarom,scriptie proces einde wetenschap rapporter ond...,Valoriseren van bevindingen (outreach),Valoriseren van bevindingen (outreach),Waarde van informatie/data
4,Hoe herken je fake news,NaN,nieuws naam democratie muziek koning mens inte...,Beoordelingscriteria relevantie/ betrouwbaarheid,Beoordelen van resultaten (kritisch),Beoordelen van resultaten (kritisch)


In [11]:
corpora = data['content'].to_list()

In [12]:
#load a set of stop words
stopwords=get_stopwords_list(STOPWORD_PATH)

# Initializing TF-IDF Vectorizer with stopwords
vectorizer = TfidfVectorizer(ngram_range=(1,1),stop_words=stopwords, smooth_idf=True, use_idf=True)

# Creating vocab with our corpora
vectorizer.fit_transform(corpora)

# Storing vocab
feature_names = vectorizer.get_feature_names()

In [13]:
result = [] #a list of dictionary, key is 'top_keywords', value is list of keywords
for doc in corpora:
    df = {}
    #df['full_text'] = doc
    df['top_keywords'] = get_keywords(vectorizer, feature_names, doc)
    result.append(df)


final = pd.DataFrame(result)
final

,top_keywords
0,"[schouder, reus, literatuur, document, plek, c..."
1,"[zoekstrategie, uitleg, tabblad, literatuurond..."
2,"[citatie, referentie, toevoeg, refwork, logboe..."
3,"[kennis, publiek, maatschappij, doelgroep, die..."
4,"[nieuws, bericht, verkiezing, leven, intentie,..."
5,"[search, mesh, subject, querie, filter, zoekop..."
6,"[informatie, vraag, waarde, proces, kanaal, fa..."
7,"[afbeelding, plaat, toestemming, licentie, rec..."
8,"[foto, film, maker, kunstwerk, toestemming, af..."
9,"[zoekbalk, optie, zoekresultaat, motivatie, le..."


In [14]:
text_keyword=[]
for item in result:
    text_keyword.append(item['top_keywords'])
print(text_keyword[:5])

[['schouder', 'reus', 'literatuur', 'document', 'plek', 'collectie', 'onderzoeker', 'vereniging', 'repositorie', 'opinie'], ['zoekstrategie', 'uitleg', 'tabblad', 'literatuuronderzoek', 'onderdeel', 'beoordeel', 'pagina', 'zoektechniek', 'zoekmethode', 'subpagina'], ['citatie', 'referentie', 'toevoeg', 'refwork', 'logboek', 'worddocument', 'verwerken', 'procesmanagement', 'kopie', 'folder'], ['kennis', 'publiek', 'maatschappij', 'doelgroep', 'dienst', 'einde', 'resultaat', 'onderzoek', 'writing', 'weg'], ['nieuws', 'bericht', 'verkiezing', 'leven', 'intentie', 'beslissing', 'arno', 'tijd', 'muziek', 'mens']]


In [15]:
# read taxonomy
import pickle
with open('../data/LO_picklefile/surf.pickle','rb') as pickle_file:
    los = pickle.load(pickle_file)

In [16]:
extracted_key_value_from_dict = []
for item in los:
    key_to_extract = {'name','keywords'}
    extracted_dict = {key: item[key] for key in item.keys()
                               & key_to_extract}
    extracted_key_value_from_dict.append(extracted_dict) 

In [17]:
from gensim.models import KeyedVectors
print("loading")
word2vec_model  = KeyedVectors.load_word2vec_format("/Users/zjy/Desktop/EDIA intern/code/w2v_model/cc.nl.300.vec")
print("done loading")

loading
done loading


In [18]:
import numpy as np

# A function that takes a sentence as a list of tokens and returns the mean vector for a given model
def get_sentence_vector(sentence, model): 
    word_vectors =[]
    for word in sentence: 
        # Ignore words that are not in the vocabulary
        try: 
            index = model.key_to_index[word]
            word_vectors.append(model.get_vector(word))
        except:
            continue
    
    word_matrix = np.asarray(word_vectors)
    mean_vector = np.mean(word_matrix, axis=0)
    sum_vector = np.sum(word_matrix, axis=0)

    return mean_vector, sum_vector

In [19]:
def classify_text(text_kws, los):
    """classify text by counting the overlapping keywords between text and taxonomy
       text_kws: list of lists, keywords extracted from text
       los: list of dicts, keywords of learning objectives
    """
    from scipy.spatial import distance
    rv = []
    for ks in text_kws:
        #initialize the value of lo['name']
        scores = {lo['name']: 0 for lo in los}
        vector1 = get_sentence_vector(ks, word2vec_model)[0]
        for lo in los:
            #create a set to collect cleaned value of dictionary
            cl=set()
            for kw in lo['keywords']:
                # clean up the value of dictionary
                cw = kw.lower().replace('_',' ')
#                 for i in nlp_nl(cw):
                cl.add(cw)
            
            if cl:
                vector2 = get_sentence_vector(cl, word2vec_model)[0]
                scores[lo['name']] = 1- distance.cosine(vector1,vector2)
            
        rv.append(scores)
            
    return rv

In [20]:
text_kws = text_keyword
los = extracted_key_value_from_dict
classified=classify_text(text_kws, los)

/Users/zjy/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/zjy/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [21]:
#sort dictionary by the number of overlapping keywords, from high to low
#def sort_dict(classify_text):
for dic in classified:
    x=sorted(dic.items(), key=lambda dic:dic[1], reverse=True)
    print(x)
    print()
    print(x[0][0])
    print()


[('Zoeken ter oriëntatie', 0.6533944010734558), ('Kritisch beoordelen', nan), ('Beoordelingscriteria (relevantie/betrouwbaarheid)', 0.6874606013298035), ('Aard en verschijningsvormen van informatie/data', 0.6791712045669556), ('Waarde van informatie/data', 0.6546018123626709), ('Beoordelen van resultaten (kritisch)', 0.6287640929222107), ('Valoriseren van bevindingen (outreach)', 0.6248579621315002), ('Publiceren (van eindproduct)', 0.5950257778167725), ('Samenwerken (in teams)', 0.5879917144775391), ('Synthetiseren of creëren van nieuwe informatie', 0.5835874676704407), ('Communiceren (van eindproduct)', 0.5758397579193115), ('Selecteren informatiebronnen en zoeksystemen', 0.5680572986602783), ('Informatieverwerking (lezen, luisteren e.d.)', 0.5451504588127136), ('Manage proces en zoekresultaten', 0.5294864773750305), ('Formuleren van onderzoeksvraag', 0.5220149159431458), ('Werking en structuur van het internet', 0.5183538198471069), ('Identificeren van informatiebehoefte', 0.5175941